**Imports**

In [1]:
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from A1_T2_Q1_2021258_Q2_2021555 import BigramLM
from transformers import pipeline

# Bigram Model

**Karan Gupta 2021258**

**Bigram Model (No Smoothing)**

In [2]:
b1=BigramLM()
b1.add_data("data\corpus.txt")
b1.learn()

**Rahul Oberoi 2021555**

**Laplace and KneserNey Smoothing**

In [3]:
# b1.KneserNey_learn()
# b1.Laplace_learn()
b1.find_top5_bigrams()

# Probability Explanation for sentences

**Karan Gupta 2021258**

In [4]:
classifier = pipeline("text-classification", model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True,device=0)
def emotion_scores(words):
    emotions = classifier(words)
    return emotions[0]

l={}
emo=['anger','fear','joy','love','sadness','surprise']
for i in emo:
    with open(f"Test/gen_{i}_Laplace.txt","r") as f:
        s=f.readlines()[0].strip()
        l[s]=i

# l={'i action abandoning adrasteius involves end joan energy officer reilly results were ranted dan lamb contracts testimony alice achieved daughter parents':"anger","i potentially negative right acts was involved crises ambulatory i reluctant afraid":"fear","i dip bloom experienced comparison everybodys tv nice f respecting beyond hang thanked":"joy","i gentle enjoyed talking relations structure romantic starved such buffed shouldnt sensed laughed hart hairstye wise swear wedding lover riders kisses beloved romantically":"love","i unfit learning emotional contain pathetic overly unsuccessful kenny but latter isolation circumstance remorse tackling toxins this beginning resigned tourists painless":"sadness","i feel surprise isolation haunt rations amazingness figs editors experiment radiant unclear stic etc swamp path unusual":"surprise"}
# l_no={'i particular smell refusal negative viewed jason i record':"anger","i potentially negative right acts was involved crises ambulatory i reluctant afraid":"fear","i radiator also feel fantastic snob wowed tell keenly i austen tales community taylor primer professional":"joy","i feel birth soothing paternal under kisses twitter im smelling flirty passionate sony":"love",}
with open("evaluation.txt","w") as f:
    for i in l:
        print(type(i))
        f.write(i+"\n")
        sentenceprob,bigramprob=b1.findsentenceprob(i,'Laplace')
        f.write(f"sentence prob:{sentenceprob} \n")
        sen_score=emotion_scores(i)
        for k in sen_score:
            if k['label']==l[i]:
                f.write(f"sentence emotion score:{k} \n")

        f.write("words emotion scores: \n")
        words=i.split()
        for j in range(len(words)-1):
            score=emotion_scores(words[j]+" "+words[j+1])
            for k in score:
                if k['label']==l[i]:
                    k['words']=words[j]+" "+words[j+1]
                    k['bigramprob']=bigramprob[j]
                    f.write(f"{k}\n")




c:\Users\gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


<class 'str'>


c:\Users\gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


# Extrinsic Evaluation

**Shivesh Gulati 2021286**

**Smoothing Technique**

In [5]:
smoothing_techniques=['No','Laplace','KneserNey']
emotions=['anger','joy','love','fear','surprise','sadness']

**Loading the saved models**

In [6]:
model,encoder,vectorizer = pickle.load(open("model.pk1","rb"))

c:\Users\gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.3.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.2 when using version 

**Encoding the emotions**

In [7]:
emotions_encoded = encoder.transform(emotions)
mapping =[]
for i in range(len(emotions)):
    mapping.append((emotions[i],emotions_encoded[i]))
print(mapping)

[('anger', 0), ('joy', 2), ('love', 3), ('fear', 1), ('surprise', 5), ('sadness', 4)]


**Loading the text files**

In [8]:
def create_test(smoothing_technique):
    x_test=[]
    y_test=[]
    for emotion in mapping:
        f = open(f"Test/gen_{emotion[0]}_{smoothing_technique}.txt")
        for line in f:
            line = line[:-1]
            x_test.append(line)
            y_test.append(emotion[1])
    x_test = vectorizer.transform(x_test)
    return [x_test,y_test]

**Getting the accuracy scores**

In [9]:
print("Metrics for the model trained :-")
for smoothing_technique in smoothing_techniques:
    x_test,y_test = create_test(smoothing_technique)
    y_test_pred = model.predict(x_test)
    print(f"\nMetrics for {smoothing_technique} Smoothing")
    print("Accuracy Score = ",accuracy_score(y_test,y_test_pred))
    print("Macro F1 Score = ",f1_score(y_test,y_test_pred,average='macro'))

Metrics for the model trained :-

Metrics for No Smoothing
Accuracy Score =  0.63
Macro F1 Score =  0.6182748978204119

Metrics for Laplace Smoothing
Accuracy Score =  0.6766666666666666
Macro F1 Score =  0.6761520851034918

Metrics for KneserNey Smoothing
Accuracy Score =  0.65
Macro F1 Score =  0.6451522232775957
